In [ ]:
%load_ext jupyternotify

In [ ]:
# 2.27.21
# This 123NF driver has all the funcs, whereas driver_code_forPaper-blank.ipynb has only the 123NF for heatmap paper

In [ ]:

import importlib
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
from graphviz import Source, render
import pydot

import datetime
import time

import sys
sys.path.append('py_modules') # below modules are in this folder
#print(sys.path)
import setup_nx # your own module, setup.nx.py
importlib.reload(setup_nx)
import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm
import my_process_funcs as prc # new


In [ ]:
# [Essential] specify input feeder data for IEEE 123-node test feeder

# Enter the path/name of the impedance model data (excel file)
filepath = "feeder_impedance_models/"
modelpath = filepath + "004_GB_IEEE123_OPAL_accur.xlsx"
#modelpath = filepath + "004_GB_IEEE123_OPAL_mod2.xlsx"

    # All GridBright load files should be in the following folder
    #loadfolder = "/Users/jasperpakshong/Documents/Berkeley/ENERGISE/IEEE13/"
    #loadpath = loadfolder + "IEEE13testload_w_extreme_act.xlsx"

    # filepath = "IEEE13/"
    # modelpath = filepath + "001 phasor08_IEEE13_OPAL.xls"
    # loadfolder = "IEEE13/"
    # loadpath = loadfolder + "001_phasor08_IEEE13_norm03_HIL_7_1.xlsx"

    #filepath = "AL0001/"
    #modelpath = filepath + "AL0001_OPAL_working.xls"
    #loadfolder = "AL0001/"
    #loadpath = loadfolder + "AL0001_tvload_afternoon1h.csv"
      
    #filepath = "13NF_balanced/"
    #modelpath = filepath + "016 GB_IEEE13_balance_all_ver2.xls"
    #loadfolder = "13NF_balanced/"
    #loadpath = loadfolder + "016 GB_IEEE13_balance all ver2_time_sigBuilder_secondWise_norm03.csv"

#==========================================================================================================

#file_name = string specifying name of dot file created when make_graph() is called
file_name = '123NF'
#file_name = 'demo123NF.dot'

# Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_150'
timesteps = 1

# initialize some variables
ts = time.time()
print(datetime.datetime.fromtimestamp(ts))
plot = 0 #turn plot on/off
depths = {}
leaves = []

In [ ]:
# [ESSENTIAL] create feeder object

fin_feeder = ff.feeder_init(modelpath,'','',timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths,file_name)

#print('depths=',depths) # check this is populated, lists how far each node is from substation
#print('depths length=',len(depths))

# print list of first 10 buses in network
print("First ten nodes are:")
count = 0 
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
import csv  # jaimie do you need this?
graph = fin_feeder.network

In [ ]:
# ------- Optional -----------------
# Run if want to diaply generated .png files in the python notebook
from IPython.display import display
from PIL import Image

# shorten png printing commands into one function call
def printpng(filename):
    im = Image.open(filename) 
    #print(im.size) # (pixel width,pixel height)
    width=im.size[0]
    height=im.size[1]
    im.draft('RGB',(width/1,height/1)) 
    px = im.load()
    display(im)
    # reference on img.draft func: https://stackoverflow.com/questions/57663734/how-to-speed-up-image-loading-in-pillow-python
    
    return

In [ ]:
# ------------ run find_good_colocated -----------------
# assessed feas of placing 1 co-located actuator at each loc and makes colormap

# all_act_locs = []
# perf_nodes = []
# here we set act_locs=[] to create heatmap for empty network
# feas_configs, heatMapNames=prc.find_good_colocated(fin_feeder, [], node_index_map, substation_name, depths,file_name, Vbase_ll, Sbase)

# print('----------- Feas configs are: ---------')
# print(feas_configs)
# Source.from_file(heatMapNames[0]) # display graph in notebook

In [ ]:
# Create figure 2, colormap of phase coupling 
ratios = vis.phaseCouplingPerNode(fin_feeder,depths,file_name)
for key, val in ratios.items():
    ratios[key] = np.mean(val) # take the mean of phase coupling across all three phases
    
vis.createColorMap(fin_feeder, ratios, 'fig2_123NF')
ff.clear_graph(fin_feeder)
Source.from_file('colorMap_fig2_123NFwide') # display graph in notebook

In [ ]:
# ----- Run computeFParamSpace, for debugging ---------
# all_act_locs = ['bus_25','bus_39','bus_56','bus_108'] # 49, then 76, then 65
# perf_nodes = ['bus_25','bus_39','bus_56','bus_108']

# Fq_ub,Fp_ub=ctrl.computeFParamSpace_v2(fin_feeder, all_act_locs, perf_nodes,R,X,depths,node_index_map)
# # this function prints Zgood (from R and X matrices) and Z_toSubst, where Z_toSubst is correct but doesnt match Zgood. Investigate
# print('(Fp_ub,Fq_ub)=(',Fp_ub,',',Fq_ub,')')

#vis.markActuatorConfig(all_act_locs, fin_feeder, file_name)

In [ ]:
# --------- Config Chunk (Assign act and perf locs here) ----------

# binary heat map that shows good locations to place a new actuator when
# a) there are 0 existing actuators, b) there is 1 existing actuator, and c) there are two existing actuators.
# actuator list = complex config 1, i.e. at each step we choose one of the good places and buildup to complex config 1

# #complex config 1 (feas)
#all_act_locs = ['bus_49','bus_76','bus_65'] # 49, then 76, then 65
#perf_nodes = ['bus_49','bus_76','bus_65']
# 16336 seconds 

#complex config 2 (feas)
# all_act_locs = ['bus_25','bus_39','bus_56','bus_108'] # 49, then 76, then 65
# perf_nodes = ['bus_25','bus_39','bus_56','bus_108']
# 30206 seconds

# #complex config 3 (feas)
# all_act_locs = ['bus_105','bus_112','bus_100','bus_91'] # 49, then 76, then 65
# perf_nodes = ['bus_105','bus_112','bus_100','bus_91']
# # time taken = 21674 seconds

# #complex config 4, many act 1 perf
#all_act_locs = ['bus_42','bus_42','bus_42','bus_42'] # 49, then 76, then 65
#perf_nodes = ['bus_151','bus_46','bus_251','bus_8']
# # time taken = XXX seconds

# # complex config 3
# all_act_locs = ['bus_105','bus_112','bus_100','bus_91'] # 49, then 76, then 65
# perf_nodes = ['bus_105','bus_112','bus_100','bus_91']

# evaluate this config
# all_act_locs = ['bus_60','bus_58','bus_20','bus_72'] # 49, then 76, then 65
# perf_nodes = ['bus_300','bus_58','bus_20','bus_72']

# Adam sim
# all_act_locs = ['bus_49','bus_76','bus_152'] # 49, then 76, then 65
# perf_nodes = ['bus_49','bus_76','bus_65']

# # eval neighborhood config
# all_act_locs = ['bus_82','bus_87','bus_76','bus_49','bus_41','bus_46','bus_60'] # 49, then 76, then 65
# perf_nodes = ['bus_77','bus_77','bus_77','bus_44','bus_44','bus_44','bus_66']

# run RHP on neighborhood config for 7th step only:
# set_act_locs = ['bus_82','bus_87','bus_76','bus_49','bus_41','bus_46']
# set_perf = ['bus_77','bus_77','bus_77','bus_44','bus_44','bus_44']
# addon_act_nodes = ['bus_60']
# addon_perf_nodes = ['bus_66']

# heatmap/collect stable configs for steps 1-7
# set_act_locs = []
# set_perf = []
# addon_act_nodes = ['bus_82','bus_87','bus_76','bus_49','bus_41','bus_46','bus_60']
# addon_perf_nodes = ['bus_77','bus_77','bus_77','bus_44','bus_44','bus_44','bus_66']

# # run RHP for step 3 of 10-step placeMaxColoc, not going in paper
# set_act_locs = ['bus_58','bus_20','bus_72']
# set_perf = ['bus_58','bus_20','bus_72']
# addon_act_nodes = ['bus_300']
# addon_perf_nodes = ['bus_300']

# motivating example (motex), list is equal to output 'max_act_config_stopInfeas'
# set_act_locs = ['bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
# set_perf = ['bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
# addon_act_nodes = ['bus_60']
# addon_perf_nodes = ['bus_60']


# Auto CPP configs:
# seed=1
# set_act_locs = []
# set_perf = []
# addon_act_nodes = ['bus_114','bus_52','bus_85','bus_8','bus_75','bus_106','bus_22','bus_113','bus_4','bus_7','bus_4','bus_3','bus_2','bus_5']
# addon_perf_nodes = ['bus_114','bus_52','bus_85','bus_8','bus_75','bus_106','bus_22','bus_113','bus_4','bus_7','bus_4','bus_3','bus_2','bus_5']

# # seed=2
# set_act_locs = []
# set_perf = []
# addon_act_nodes = ['bus_96','bus_85','bus_97','bus_83','bus_105','bus_11','bus_152','bus_197','bus_6','bus_102','bus_58','bus_4','bus_2']
# addon_perf_nodes = ['bus_96','bus_85','bus_97','bus_83','bus_105','bus_11','bus_152','bus_197','bus_6','bus_102','bus_58','bus_4','bus_2']

# seed=3
# set_act_locs = []
# set_perf = []
# addon_act_nodes = ['bus_197','bus_55','bus_5','bus_113','bus_33','bus_96','bus_45','bus_63','bus_59','bus_106','bus_10','bus_91','bus_46','bus_25','bus_6','bus_17','bus_80','bus_49','bus_4','bus_73',]
# addon_perf_nodes = ['bus_96','bus_85','bus_97','bus_83','bus_105','bus_11','bus_152','bus_197','bus_6','bus_102','bus_58','bus_4','bus_2']

In [ ]:
# ---------- mark actuator config -------------
# assign 'all_act_locs' in config chunk
vis.markActuatorConfig(all_act_locs,fin_feeder, file_name)

# convert the generated .dot to.png:
# (graph,) = pydot.graph_from_dot_file('generated_figs/'+'actConfig_'+ file_name) # requires 'import pydot'
# graph.write_png('generated_figs/'+'config_60.png') # can change the name
# printpng('generated_figs/config_60.png') # can change the name

In [ ]:
# --------- run eval_config on CPP config --------------
# need check that end-config of CPP process is feasible for both control frameworks

parmObj=hm.configParms()
# parmObj.set_ctrlTypes(['VVC', 'VVC', 'VWC', 'VWC', 'VWC', 'VVC', 'VVC', 'VVC'])
# parmObj.set_version(2) # 1 for PBC

parmObj.set_ctrlTypes(['PBC', 'PBC', 'PBC', 'PBC', 'PBC', 'PBC', 'PBC', 'PBC'])
parmObj.set_version(1) # 1 for PBC


#all_act_locs = ['bus_152', 'bus_21', 'bus_58', 'bus_70', 'bus_4', 'bus_82', 'bus_109', 'bus_47']
# all_act_locs = ['bus_47','bus_58','bus_4','bus_109','bus_21','bus_70','bus_152','bus_82']
# perf_nodes=['bus_47','bus_58','bus_4','bus_109','bus_21','bus_70','bus_152','bus_82']
# print(len(all_act_locs))
# print(len(foo))
feas, maxError,numfeas,bestF,indicMat=prc.eval_config(parmObj,fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase)



In [ ]:
# -------- run eval_config for a config with actuators at ALL nodes of the network  -----------
# from running this, hypothesis is: multiple actautors on every node tracking single perf is feas for ANY choice of perf

# assign 'all_act_locs' and 'perf_nodes' in config chunk
graphNodes_nosub = hm.remove_subst_nodes(fin_feeder, file_name) # dont consider co-located at substation nodes, node 650 and 651
all_act_locs=graphNodes_nosub[:] # place actuators at ALL non-substation nodes
n=len(all_act_locs)
print('acts=',all_act_locs)

perf='bus_84' # you choose
perf_nodes=[perf] * n
print('perfs=',perf)
feas, maxError,numfeas,indicMat=prc.eval_config(parmObj,fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase)

#printpng('generated_figs/heatmap_colocated_123NF.png')

In [ ]:
# ----- good vs. bad branch scenario, 1-step heatmap
set_acts = ['bus_8','bus_53','bus_57','bus_66'] # has bad branch
addon_acts= ['bus_152'] # arbitrary

parmObj=hm.configParms()
foo=['VVC','VWC','VVC','VWC']
parmObj.set_ctrlTypes(foo)
parmObj.set_version(2) # 1 for PBC

# here we set act_locs as the existing actuators
feas_configs, heatMapNames=prc.find_good_colocated(parmObj,fin_feeder, set_acts, addon_acts, node_index_map, substation_name, depths,file_name, Vbase_ll, Sbase)

In [ ]:
# ----- good vs. bad branch scenario, 1-step heatmap
set_acts = ['bus_8','bus_53','bus_57','bus_74'] # has good branch
addon_acts= ['bus_152'] # arbitrary

parmObj=hm.configParms()
foo=['VVC','VWC','VVC','VWC']
parmObj.set_ctrlTypes(foo)
parmObj.set_version(2) # 1 for PBC

# here we set act_locs as the existing actuators
feas_configs, heatMapNames=prc.find_good_colocated(parmObj,fin_feeder, set_acts, addon_acts,node_index_map, substation_name, depths,file_name, Vbase_ll, Sbase)

# result looks like: 'heatmap_colocated' + '_' + file_name

In [ ]:
# ---------------- run CPP (formely called find_good_coloated) ---------------------------
parmObj=hm.configParms()
parmObj.set_ctrlTypes(['VVC', 'VVC', 'VWC', 'VWC', 'VWC', 'VVC', 'VVC', 'VVC']) # format is ctrl types of [set_acts addon_acts]
parmObj.set_version(2) # 1 for PBC

# parmObj.set_ctrlTypes(['PBC', 'PBC', 'PBC', 'PBC', 'PBC', 'PBC', 'PBC', 'PBC'])
# parmObj.set_version(1) # 1 for PBC

# set_acts = []
# addon_acts=['bus_152', 'bus_21', 'bus_58', 'bus_70', 'bus_4', 'bus_82', 'bus_109', 'bus_47']
set_acts = []
addon_acts = ['bus_47','bus_58','bus_4','bus_109','bus_21','bus_70','bus_152','bus_82']


t = time.time()
lst_feas_configs, lzn_error_run_sum, heatMapNames = prc.find_good_colocated(parmObj,fin_feeder, set_acts, addon_acts, node_index_map,substation_name, depths, file_name, Vbase_ll, Sbase)
# lst_feas_configs is list of dictionaries
elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
## ------------ run placeMaxColocActs_stopAtInfeas, aka overloadCPP process -----------
# place colocated actuators until an infeasible loc is tested, then call find_good_colocated and return 
# produces colormap associated with "motex" motivating example

t = time.time()
parmObj=hm.configParms()
parmObj.set_version(2) # 1 for PBC
max_act_config_stopInfeas,parmObj=prc.placeMaxColocActs_stopAtInfeas(parmObj,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase)
elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
# ------ run place_max_colocated_acts, aka auto_overloadCPP process -------------
t = time.time()
seed=2 # choose
parmObj=hm.configParms()
parmObj.set_version(2) # 1 for PBC
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config,parmObj = prc.place_max_coloc_acts(parmObj,seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase)
elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
# ------- run runHeatMapProcess (RHP) ---------------
# assign 'set_act_locs, set_perf, addon_locs, addon_perf_nodes' in config chunk
# set_act_locs and set_perf are existing pairs, addon_locs and addon_perf_nodes are pairs to add after produce heatmap


# TEMP - REMOVE BELOW CHUNK SOON:
# # seed=2
set_act_locs = []
set_perf = []
# addon_act_nodes = ['bus_96','bus_85','bus_97','bus_83','bus_105','bus_11','bus_152','bus_197','bus_6','bus_102','bus_58','bus_4','bus_2']
# addon_perf_nodes = ['bus_96','bus_85','bus_97','bus_83','bus_105','bus_11','bus_152','bus_197','bus_6','bus_102','bus_58','bus_4','bus_2']
addon_act_nodes = ['bus_96','bus_85','bus_97','bus_83','bus_11','bus_108','bus_197','bus_59','bus_102','bus_7']
addon_perf_nodes = ['bus_96','bus_85','bus_97','bus_83','bus_11','bus_108','bus_197','bus_59','bus_102','bus_7']
# bus_2 is the last one of the CPP, after bus_7

# lets assume the VVC and VWC were chosen randomly during the auto-CPP, resulting in:
parmObj=hm.configParms()
parmObj.set_ctrlTypes(['VWC','VWC','VVC','VWC','VVC','VVC','VVC','VVC','VVC','VVC'])
parmObj.set_version(2) # 1 for PBC


t = time.time()
lst_feas_configs, lzn_error_run_sum, heatMapNames = prc.runHeatMapProcess(parmObj,fin_feeder, set_act_locs, set_perf, addon_act_nodes, addon_perf_nodes, node_index_map, substation_name, depths, file_name,Vbase_ll, Sbase)
# lst_feas_configs is list of dictionaries
elapsed = time.time() - t
print('Time elapsed=',elapsed)

# convert .dot to .png
# (graph,) = pydot.graph_from_dot_file('generated_figs/'+'NPP_heatmap_step0_' + file_name) # requires 'import pydot'
# graph.write_png('generated_figs/'+'RHP_nbhd_2.27.png') # can change name
# printpng('generated_figs/RHP_nbhd_2.27.png')

In [ ]:
# # Save RHP variable 'lst_feas_configs' to .pkl file, so dont have to run again in future
import pickle
filename='CPPseed2_lstfeasconfigs_droop_mod1.6.pkl'
with open(filename, 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(lst_feas_configs, f)

In [ ]:
# ---------- Setup branch analysis on 'lst_feas_configs' populated by RHP -------------

In [ ]:
# load the data 'lst_feas_configs' from .pkl file
# import pickle
# filename='nbhs_RHP_lstfeasconfigs.pkl'
# with open(filename,'rb') as f:  # Python 3: open(..., 'rb')
#     lst_feas_configs = pickle.load(f)
     
    
import os
import pickle
scores = {} # scores is an empty dict already
filename='CPPseed2_lstfeasconfigs_PBC_mod1.1.pkl'
print(os.path.getsize(filename))
if os.path.getsize(filename) > 0:      
    with open(filename, "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        lst_feas_configs = unpickler.load()
        
print(len(lst_feas_configs)) # just to see if it loaded

In [ ]:
# to save and restore variables..
# https://stackoverflow.com/questions/2960864/how-can-i-save-all-the-variables-in-the-current-python-session

In [ ]:
# ------------ create histogram of numfeas --------------
numFeasList=[]
#print([d['numfeas'] for d in lst_feas_configs])
for cfg in lst_feas_configs:
    a=cfg['numfeas']
    b=a[0].item() # remove layers of brackets
    numFeasList.append(b)

#print(numFeasList)
plt.title("numFeas Histogram")
plt.xlabel("number of feasible Fs")
plt.ylabel("number of configs")
plt.hist(numFeasList, bins=40) # -10: is last 10 ele
#plt.hist(numFeasList[-10:], bins=40) # -10: is last 10 ele
plt.savefig('numFeasHist.png') # modify for each feeder
plt.show() # need to savefig before plt.show

print(len(numFeasList))

In [ ]:
# convert lst_feas_configs (list of dictionaries) to newlst_feas_configs (a list of lists)
# input a list of feasible actuator configurations where each configuration is its own list (ie a list of lists)"
length=len(lst_feas_configs)
newlst_feas_configs = []
numact_lst=[] # store number of actuators in separate list
i=0
for dic in lst_feas_configs:
    #print('dic=',dic)
    #print(type(dic))
    i+=1
    if i<length-1:
        #print('dic[act]=',str(dic['act']))
        newlst_feas_configs += [dic['act']]
        numact_lst+=[dic['numfeas']]

In [ ]:
# create 'placed' variable
# placed is an ordered list of lists, where the k'th ele of the list is the actuators placed at that step
set_acts=[]
addon_acts=['bus_152', 'bus_21', 'bus_58', 'bus_70', 'bus_4', 'bus_82', 'bus_109', 'bus_47']
acts=set_acts + addon_acts

placed=[]
k=0
while k<=len(acts):
    placed+=[acts[0:k]]
    k+=1
print(placed)

In [ ]:
# ----- run plot_actuator_num_histogram ---------"
# input list of lists of feasible actuator configurations"
# output histogram reflecting number of actuators in each feas configuration"
vis.plot_actuator_num_histogram(newlst_feas_configs, file_name)  
# note the max number in the histogram

# ----- run assign_network_branches ---------"
branches = vis.assign_network_branches(fin_feeder, substation_name)
vis.mark_network_branches(fin_feeder, branches, file_name, substation_name,depths)
#Source.from_file('branch_key:' + file_name)
print('branches table:') # format into a table with index and branch nodes
i=0
for br in branches:
    print('idx',i,': ',br)
    #print('phase',vis.getPhases(br,fin_feeder))
    i+=1


# ----- run find_good_branches ---------"
# the 'best' branch is the branch with the most actuators from feas configs on it across all configurations
num_good_bad_branches = 4 # return the n best and worst branches
vis.find_good_branches(newlst_feas_configs, branches, num_good_bad_branches, placed)

# ----- run determine_good_or_bad_branch ---------"
#print('\nBranches are ', branches)
branch = branches[-2]  # you choose as input
print('\nBranch chosen=',branch)
good_branch_threshold = 3 # threshhold # configs that must use the branch (i.e. an act on that branch) for the branch to be "good"
configs_with_branch = vis.determine_good_or_bad_branch(branch, newlst_feas_configs, good_branch_threshold)
# output: configs that use the branch, and if that # exceeds threshhold, call the branch "good"
print('Num feas configs =', len(newlst_feas_configs))

In [ ]:
# -------- Example experiment for Table 1; phase ratio mod2x, PBC, m=5 -----------
# Desc: find an $m$-node configuration under PBC or droop control that is \emph{good} on the orig 123NF 
# and record the best associated $F$. Then we apply this best $F$ to the same configuration 
# on a modified 123NF, where every line has a higher R/X or phase ratio with the same line length
#If the configuration on the higher-ratio 123NF is no longer \emph{good}, the result supports the remarks. 
# Then we do the reverse.

i=0
support=0 # count how many times inc feeder ratio loses stability for same config/kgain set
not_support=0
while i<15: # try design config for a number of trials
    print('-----------------------------')
    
    # --------- Initialize orig feeder -------------
    modelpath = filepath + "004_GB_IEEE123_OPAL.xls"
    file_name = '123NForig'
    fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
    node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix

    parmObj=hm.configParms()
    parmObj.set_version(1) # 1 for PBC
    numAct=5
    seedkey=i # different random number seed for each trial, so get different sets of random configs
    parmObj,act_locs,bestFparm,indicMat=prc.design_config(parmObj,seedkey,numAct,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase)
    
    if bestFparm is not -1: # proceed if designing config found a config
        # --------- Initialize mod feeder -------------
        modelpath = filepath + "123NF_phrat_mod2.xls"
    #     modelpath = filepath + "123NF_phrat_mod1.5.xls"
    #     modelpath = filepath + "123NF_rxrat_mod2.xls"
    #     modelpath = filepath + "123NF_rxrat_mod1.5.xls"
        file_name = '123NFmod2'
        fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
        node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix

        feas=ctrl.eval_config_F(parmObj,bestFparm,indicMat,fin_feeder,depths,node_index_map)
        if feas==False: # if bestF on mod feeder is infeas, break
            print('-----> Support! F on modded feeder is infeas')
            support+=1
        else:
            print('-----> inconclusive, F on modded feeder is still feas')

        
    # now do the reverse...
    parmObj,act_locs,bestFparm,indicMat=prc.design_config(parmObj,seedkey,numAct,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase)
    
    if bestFparm is not -1: # proceed if designing config found a config
        print(act_locs)  # print config found
        print(parmObj.get_ctrlTypes())

        # --------- Initialize orig feeder -------------
        modelpath = filepath + "004_GB_IEEE123_OPAL.xls"
        file_name = '123NForig'
        fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
        node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix

        feas=ctrl.eval_config_F(parmObj,bestFparm,indicMat,fin_feeder,depths,node_index_map)
        if feas==False: # if bestF on mod feeder is infeas, break
            print('-----> Not support - F on orig feeder is infeas')
            not_support+=1
        else:
            print('-----> inconclusive, F on orig feeder is still feas')
    
    if support+not_support>=10:
        break
        
    i+=1
#----------------------------------------------------------------------------
# Print results
print('total number evaluated:',i+1)
print('support=',support)
print('not support=',not_support)
try: 
    percent_support=support/(support+not_support)
except: 
    print('error: support+not_support = 0')

print('percent_support=',percent_support)

In [ ]:
# --------- TEMP, test out new Gdisk computF_parm and assignF --------------
# Gdisk sandbox code is in dir: C:\Users\jaimi\Documents\MATLAB\Research\topology_work

In [ ]:
# # # Save RHP variable 'indicMat' to .pkl file, so dont have to run again in future
# import pickle
# filename='indicMat_8.25.pkl'
# with open(filename, 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(indicMat, f)
    

In [ ]:
# load indicMat
import os
import pickle
scores = {} # scores is an empty dict already
filename='indicMat_8.25.pkl'
print(os.path.getsize(filename))
if os.path.getsize(filename) > 0:      
    with open(filename, "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        indicMat = unpickler.load()
        
print(len(indicMat)) # just to see if it loaded

In [ ]:
%%notify
# ------ run detControlMatExistence ---------
t = time.time()

parmObj=hm.configParms()
parmObj.set_ctrlTypes(['PBC'])
parmObj.set_version(1) # 1 for PBC

n=int(len(indicMat)/6)
A,B=hm.setupStateSpace(parmObj,fin_feeder, n,node_index_map, depths)
t = time.time()
#f_ub_list=ctrl.computeFParamSpace_v3(B,indicMat)
feas,feasFs,numfeas,numTried,num_act,bestF,indicMat=ctrl.detControlMatExistence(parmObj, A, B, indicMat)
elapsed = time.time() - t
print('Time elapsed=',elapsed)

In [ ]:
%%notify
# ------ run eval_config ---------
parmObj=hm.configParms()
# parmObj.set_ctrlTypes(['VVC', 'VVC', 'VWC', 'VWC', 'VWC', 'VVC', 'VVC', 'VVC'])
# parmObj.set_version(2) # 1 for PBC

parmObj.set_ctrlTypes(['PBC', 'PBC', 'PBC'])
parmObj.set_version(1) # 1 for PBC

#all_act_locs = ['bus_152', 'bus_21', 'bus_58', 'bus_70', 'bus_4', 'bus_82', 'bus_109', 'bus_47']
all_act_locs = ['bus_47','bus_58','bus_4']
perf_nodes=['bus_47','bus_58','bus_4']

# print(len(all_act_locs))
# print(len(foo))
feas, maxError,numfeas,bestF,indicMat=prc.eval_config(parmObj,fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase)

In [ ]:
# next steps:
# use N2 to try different perturbations in same percent level
# decide what it means for config to be 'feasible' now

In [ ]:
# apply a terrible config and see how many of the 15 work
parmObj=hm.configParms()
parmObj.set_ctrlTypes(['PBC', 'PBC', 'PBC'])
parmObj.set_version(1) # 1 for PBC
all_act_locs = ['bus_59','bus_13','bus_53']
perf_nodes=['bus_52','bus_60','bus_40']
feas, maxError,numfeas,bestF,indicMat=prc.eval_config(parmObj,fin_feeder, all_act_locs,perf_nodes, node_index_map,substation_name,depths,file_name,Vbase_ll, Sbase)